In [59]:
%reset -f

import aiohttp
import asyncio
from columnar import columnar
from bs4 import BeautifulSoup
from configparser import ConfigParser

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

''' Latest scraping version functions'''
async def fetch(url):
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=False)) as session:
        async with session.get(url) as response:
            return await response.text()
        
async def crawl(parsers, url_type):
    output = []

    used_tools = {k:v for k,v in parsers.items(url_type)}
    
    urls = list(used_tools.values())
    tools = list(used_tools.keys())
    
    tasks = [fetch(url) for url in urls]
    
    html_pages = await asyncio.gather(*tasks)
      
    for i, html in enumerate(html_pages):
        tag_n = eval(parsers['PARSERS'][tools[i] + "_tag_n"])
        tag_p = eval(parsers['PARSERS'][tools[i] + "_tag_p"])
        tag_s = eval(parsers['PARSERS'][tools[i] + "_tag_s"])
        attr_n = eval(parsers['PARSERS'][tools[i] + "_attr_n"])
        attr_p = eval(parsers['PARSERS'][tools[i] + "_attr_p"])
        attr_s = eval(parsers['PARSERS'][tools[i] + "_attr_s"])

        soup = BeautifulSoup(html, 'lxml')
        name = soup.find_all(tag_n, **attr_n)
        price = soup.find_all(tag_p, **attr_p)
        stock = soup.find_all(tag_s, **attr_s)
        
        for j in range(len(name)):
            output.append([name[j].get_text().strip()[:55], price[j].get_text().strip(), stock[j].get_text().strip()])

        headers = ['card', 'price', 'stock']
        table = columnar(output, headers, no_borders=True, max_column_width=None)

        print(table)
        # print(bcolors.BOLD + name[0].get_text() + bcolors.ENDC)
        # print(bcolors.BOLD + price[0].get_text() + bcolors.ENDC)
    
async def main():  
    parsers = ConfigParser(delimiters=(':'))
    parsers.read('../parsers.ini')
        
    '''Getting latest versions from webs'''
    await crawl(parsers, 'URLS')
        
await main()

        
  CARD                                                     PRICE      STOCK    
    
  MSI GeForce RTX 5070 Ti 16G INSPIRE 3X OC PLUS | Tarjet  1.109,90€  Agotado  
  MSI GeForce RTX 5070 Ti 16G VANGUARD SOC LAUNCH EDITION  1.149,90€  Agotado  
  MSI GeForce RTX 5070 Ti 16G GAMING TRIO OC PLUS | Tarje  1.114,90€  Agotado  
  MSI GeForce RTX 5070 Ti 16G VENTUS 3X OC | Tarjeta Gráf  889,90€    Agotado  

